In [14]:
import copy

In [15]:
class GraphNode(object):
    """
    Graph Node in a directed graph
    """
    def __init__(self, name=""):
        """
        Construct a new node, and initialize the list of parents and children.
        Each parent/child is represented by a (key, value) pair in dictionary, 
        where key is the parent/child's name, and value is an GraphNode object.
        Args:
        name: node name (string).
        """
        self.name = name
        self.parents = dict()
        self.children = dict()

    def add_parent(self, parent):
        """
        Args:
        parent: GraphNode object.
        """
        if not isinstance(parent, GraphNode):
            raise ValueError("Parent must be an instance of GraphNode class.")
        pname = parent.name
        self.parents[pname] = parent

    def add_child(self, child):
        """
        Args:
        child: GraphNode object.
        """
        if not isinstance(child, GraphNode):
            raise ValueError("Parent must be an instance of GraphNode class.")
        cname = child.name
        self.children[cname] = child

In [16]:
class BN(object):
    """
    Bayesian Network Class
    """
    def __init__(self):
        """
        Initialize list of nodes in the graph.
        Each node is represented by a (key, value) pair in dictionary, 
        where key is the node's name, and value is an GraphNode object
        """
        self.nodes = dict()

    def add_edge(self, edge):
        """
        Add a directed edge to the graph.
        
        Args:
        edge: a tuple (A, B) representing a directed edge A-->B,
            where A, B are two strings representing the nodes' names
        """
        (pname, cname) = edge

        ## construct a new node if it doesn't exist
        if pname not in self.nodes:
            self.nodes[pname] = GraphNode(name=pname)
        if cname not in self.nodes:
            self.nodes[cname] = GraphNode(name=cname)

        ## add edge
        parent = self.nodes.get(pname)
        child = self.nodes.get(cname) 
        parent.add_child(child)
        child.add_parent(parent)

    def print_graph(self):
        """
        Visualize the current BN graph.
        """
        print("Bayes Network: ")
        for node_name, node in self.nodes.items():
            print("\tNode " + node_name)
            print("\tParents: " + str(node.parents.keys()))
            print("\tChildren: " + str(node.children.keys()))

    def find_ancestors(self, observed):
        """
        Traverse the graph, find all nodes that have observed descendants.
        Args:
            observed: a list of strings, names of the observed nodes. 
        Returns:
            a list of strings for the nodes' names for all nodes 
            with observed descendants.
        """
        visit_nodes = copy.copy(observed) ## nodes to visit
        obs_ancestor = set() ## observed nodes and their ancestors

        ## repeatedly visit the nodes' parents
        while len(visit_nodes) > 0:
            next_node = self.nodes[visit_nodes.pop()]
            ## add its' parents
            for parent in next_node.parents:
                obs_ancestor.add(parent)

        return obs_ancestor

    def d_separated(self, X, E):
        """
        Find all reachable node from starting node X and rest
        of the nodes are marked d-separated from X.
        Reference : http://pgm.stanford.edu/Algs/page-75.pdf
        Args:
            X: source node
            E: evidence set, a set of observed nodes 
        """

        ## find all ancestors for E
        ancestors = self.find_ancestors(E)

        ## Try all active paths starting from the source node "X".
        ## In order to deal with v-structures, 
        ## we need to keep track of the direction of traversal:
        ## "up" if traveled from child to parent, and "down" otherwise.
        via_nodes = [(X, "up")]
        visited = set() ## keep track of visited nodes to avoid cyclic paths
        d_separate = set() ## set of nodes d_seperarated from "X"
        reachable = set() ## set of nodes reacahble from "X"

        while len(via_nodes) > 0: 
            (node_name, direction) = via_nodes.pop()
            node = self.nodes[node_name]

            ## skip visited nodes
            if (node_name, direction) not in visited:
                visited.add((node_name, direction))
                
                ## if reaches the node "end", then it is not d-separated
                if node_name not in E and (node_name) not in reachable:
                    reachable.add((node_name))
                
                ## if traversing from children, then it won't be a v-structure
                ## the path is active as long as the current node is unobserved
                if direction == "up" and node_name not in E:
                    for parent in node.parents:
                        via_nodes.append((parent, "up"))
                    for child in node.children:
                        via_nodes.append((child, "down"))
                ## if traversing from parents, then need to check v-structure
                elif direction == "down":
                    ## path to children is always active
                    if node_name not in E: 
                        for child in node.children:
                            via_nodes.append((child, "down"))
                    ## check node for v-structure
                    if node_name in E or node_name in ancestors: 
                        for parent in node.parents:
                            via_nodes.append((parent, "up"))
        
        # node not reachable from X are marked as d-separated from X
        for node_name, node in self.nodes.items():
            if node_name not in reachable and node_name not in E:
                d_separate.add((node_name))        
                         
        return d_separate, reachable

In [17]:

## Read from stdin
nedge = input("No. of edges: ")

edges = []
for line in range(int(nedge)):
    edge = input("Enter Edge nodes (A B for A -> B): ").split()
    edges += [edge]

No. of edges: 13
Enter Edge nodes (A B for A -> B): A D
Enter Edge nodes (A B for A -> B): B D
Enter Edge nodes (A B for A -> B): D G
Enter Edge nodes (A B for A -> B): D H
Enter Edge nodes (A B for A -> B): G K
Enter Edge nodes (A B for A -> B): H K
Enter Edge nodes (A B for A -> B): H E
Enter Edge nodes (A B for A -> B): C E
Enter Edge nodes (A B for A -> B): E I
Enter Edge nodes (A B for A -> B): I L
Enter Edge nodes (A B for A -> B): F I
Enter Edge nodes (A B for A -> B): F J
Enter Edge nodes (A B for A -> B): J M


In [18]:
edges

[['A', 'D'],
 ['B', 'D'],
 ['D', 'G'],
 ['D', 'H'],
 ['G', 'K'],
 ['H', 'K'],
 ['H', 'E'],
 ['C', 'E'],
 ['E', 'I'],
 ['I', 'L'],
 ['F', 'I'],
 ['F', 'J'],
 ['J', 'M']]

In [19]:
## create BN 
myBN = BN()
for edge in edges:
    myBN.add_edge(edge)

Problem 2. Finding d-separated nodes. (28 points)
Verify your solutions in HW1 problem 3 using your algorithm. More specifically:
• For problems 3(a)–3(e), find all nodes that are d-separated from the source node using your algorithm,
and find out whether the target node is included in the predicted set Y.
• For parts 3(f)–3(g), compare the nodes obtained from your algorithm with your original solution.

3.(a) Given Z = {G, L}, can influence flow from node A to node J? Justify your answer.

In [20]:
print("## D separation ##")
X = input("Starting node: ")
E = input("Evidence set (enter nodes splited by space): ").split()

d_sep, reach = myBN.d_separated(X, E)
print("Nodes d-separated from " + X + " : " + str(d_sep))

## D separation ##
Starting node: A
Evidence set (enter nodes splited by space): G L
Nodes d-separated from A : set()


As observed from the output for the above Cell, none of the nodes are d-separated from A given Z = {G, L}.

Hence, there should exist an Active trail between A and J to flow influence.

(b) Given Z = {L}, can influence flow from node A to node C? Justify your answer

In [21]:
print("## D separation ##")
X = input("Starting node: ")
E = input("Evidence set (enter nodes splited by space): ").split()

d_sep, reach = myBN.d_separated(X, E)
print("Nodes d-separated from " + X + " : " + str(d_sep))

## D separation ##
Starting node: A
Evidence set (enter nodes splited by space): L
Nodes d-separated from A : set()


As observed from the output for the above Cell, none of the nodes are d-separated from A given Z = {L}.

Hence, there should exist an Active trail between A and C to flow influence.

(c) Given Z = {D}, can influence flow from node G to node L? Justify your answer.

In [22]:
print("## D separation ##")
X = input("Starting node: ")
E = input("Evidence set (enter nodes splited by space): ").split()

d_sep, reach = myBN.d_separated(X, E)
print("Nodes d-separated from " + X + " : " + str(d_sep))

## D separation ##
Starting node: G
Evidence set (enter nodes splited by space): D
Nodes d-separated from G : {'H', 'L', 'E', 'C', 'M', 'B', 'F', 'A', 'I', 'J'}


Target node L is present in set d-separated nodes and hence, influence can not flow from G to L for given evidence set.

(d) Given Z = {D, K, M}, can influence flow from node G to node L? Justify your answer.

In [23]:
print("## D separation ##")
X = input("Starting node: ")
E = input("Evidence set (enter nodes splited by space): ").split()

d_sep, reach = myBN.d_separated(X, E)
print("Nodes d-separated from " + X + " : " + str(d_sep))

## D separation ##
Starting node: G
Evidence set (enter nodes splited by space): D K M
Nodes d-separated from G : {'C', 'B', 'J', 'F', 'A'}


Yes, influence can flow from G to L, L is not present in set of d-separated nodes from G given {D, K, M}.

(e) Given Z = {C, G, L}, can influence flow from node B to node F? Justify your answer.

In [24]:
print("## D separation ##")
X = input("Starting node: ")
E = input("Evidence set (enter nodes splited by space): ").split()

d_sep, reach = myBN.d_separated(X, E)
print("Nodes d-separated from " + X + " : " + str(d_sep))

## D separation ##
Starting node: B
Evidence set (enter nodes splited by space): C G L
Nodes d-separated from B : set()


As observed from the output for the above Cell, none of the nodes are d-separated from B given Z = {C, G, L}

Hence, there should exist an Active trail between node B and node F to flow influence.

(f) Find the set Y that contains all nodes that are d-separated from node A, given Z = {K, E}.

In [25]:
print("## D separation ##")
X = input("Starting node: ")
E = input("Evidence set (enter nodes splited by space): ").split()

d_sep, reach = myBN.d_separated(X, E)
print("Nodes d-separated from " + X + " : " + str(d_sep))

## D separation ##
Starting node: A
Evidence set (enter nodes splited by space): K E
Nodes d-separated from A : {'L', 'M', 'J', 'F', 'I'}


(g) Find the set Y that contains all nodes that are d-separated from node B, given Z = {L}.

In [26]:
print("## D separation ##")
X = input("Starting node: ")
E = input("Evidence set (enter nodes splited by space): ").split()

d_sep, reach = myBN.d_separated(X, E)
print("Nodes d-separated from " + X + " : " + str(d_sep))

## D separation ##
Starting node: B
Evidence set (enter nodes splited by space): L
Nodes d-separated from B : set()
